In [1]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import unicodedata
import re

In [2]:
data=pd.read_excel('赛题1数据材料/赛题1数据集/专利.xlsx',index_col=0)

In [3]:
data

,专利类型,授权公告日,申请日
企业编号,,,
1001,发明公布,2006-09-08­,2008-03-12
1001,发明公布,2006-06-23­,2007-12-26
1001,发明公布,2006-06-23,2007-12-26
1001,发明公布,2006-09-08,2008-03-12
1002,实用新型,2010-09-26,2011-10-26
1002,发明公布,2010-04-16,2010-09-22
1003,实用新型,2014-11-28,2015-04-22
1003,发明公布,2012-09-18,2012-12-19
1003,实用新型,2009-06-09,2010-04-14


In [4]:
data.drop_duplicates(keep='first',inplace=True)

In [5]:
data=data.replace(['发明公布更正','实用新型更正','外观设计更正','发明授权更正'],np.nan)
data=data.dropna()
count=pd.DataFrame(data.index.value_counts()).T.reindex(columns=list(range(1001,4001))).fillna(0)
count.index=pd.Index(['专利_count'])
data_1=pd.get_dummies(data['专利类型'],prefix='专利类型')
data_1=data_1.groupby('企业编号').sum().T

In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d.+\d', cell)
    if not _:
        return None
    cell = _[0]
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [7]:
data_2=data[['授权公告日']].applymap(to_day).groupby('企业编号').mean().T
data_3=data[['申请日']].applymap(to_day).groupby('企业编号').mean().T

In [8]:
data=pd.concat([data_1,data_2,data_3,count])
data=data.reindex(columns=list(range(1001,4001))).fillna(0)
data.to_pickle('专利.pickle')
data

,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
专利类型_发明专利,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
专利类型_发明公布,4.0,1.0,109.000000,1.000000,2.000000,7.000000,634.000000,155.000000,0.0,13.000000,...,2.000000,42.000000,17.000000,16.000000,27.000000,9.000000,2.000000,7.000000,5.000000,1.000000
专利类型_发明授权,0.0,0.0,59.000000,0.000000,1.000000,4.000000,241.000000,65.000000,0.0,2.000000,...,21.000000,102.000000,25.000000,41.000000,180.000000,27.000000,4.000000,16.000000,22.000000,14.000000
专利类型_外观设计,0.0,0.0,314.000000,4.000000,0.000000,2.000000,1.000000,0.000000,0.0,1.000000,...,13.000000,4.000000,2.000000,15.000000,37.000000,18.000000,0.000000,6.000000,0.000000,18.000000
专利类型_实用新型,0.0,1.0,700.000000,1.000000,3.000000,6.000000,508.000000,127.000000,2.0,1.000000,...,8.000000,36.000000,1.000000,6.000000,48.000000,16.000000,6.000000,0.000000,15.000000,1.000000
授权公告日,-4599.5,-3163.5,-2426.115905,-1311.833333,-2269.166667,-1445.421053,-1613.704480,-1177.634006,-504.0,-1902.352941,...,-1760.750000,-3254.250000,-2435.466667,-2418.692308,-2042.308219,-2212.285714,-2724.583333,-2849.310345,-2850.119048,-2683.941176
申请日,-4048.5,-2886.5,-2185.579526,-1107.166667,-1962.500000,-1066.631579,-1340.203757,-909.253602,-252.5,-1600.294118,...,-1097.068182,-2557.592391,-1756.533333,-1817.923077,-1375.493151,-1597.485714,-2279.333333,-2314.758621,-2390.857143,-2171.294118
专利_count,4.0,2.0,1182.000000,6.000000,6.000000,19.000000,1384.000000,347.000000,2.0,17.000000,...,44.000000,184.000000,45.000000,78.000000,292.000000,70.000000,12.000000,29.000000,42.000000,34.000000


In [9]:
data.index

Index(['专利类型_发明专利', '专利类型_发明公布', '专利类型_发明授权', '专利类型_外观设计', '专利类型_实用新型',
       '授权公告日', '申请日', '专利_count'],
      dtype='object')